## 实现LSTM

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

In [ ]:
# !pip install zh-dataset-inews

In [ ]:
from zh_dataset_inews import title_train, label_train, content_train, title_test, label_test, content_test

In [ ]:
len(title_train)

In [ ]:
title_train[:10]

In [ ]:
content_train[:10]

In [ ]:
label_train[:10]

In [ ]:
for x, y in zip(title_train[:10], label_train[:10]):
    print(x, y)

In [ ]:
text_vector = tf.keras.layers.experimental.preprocessing.TextVectorization()
# 学习词表
# text_vector.adapt(content_train)
text_vector.adapt(title_train)


In [ ]:
# inputs = text_vector(content_train)
inputs = text_vector(title_train)

In [ ]:
vocabs = text_vector.get_vocabulary()

vocab_size = len(vocabs)
vocab_size

In [ ]:
input_label =  tf.convert_to_tensor(label_train)

In [ ]:
embedding_dim = 1024

# model = tf.keras.Sequential([
#     # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
#     tf.keras.layers.Embedding(vocab_size, embedding_dim),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
# #    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     # use ReLU in place of tanh function since they are very good alternatives of each other.
#     tf.keras.layers.Dense(embedding_dim, activation='relu'),
#     # Add a Dense layer with 6 units and softmax activation.
#     # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
#     tf.keras.layers.Dense(6, activation='softmax')
# ])


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.summary()

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
num_epochs = 10
history = model.fit(
    x = inputs, 
    y = input_label, 
    epochs = num_epochs, 
    validation_split = 0.2,
#     verbose = 2
)

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()


plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
input_title_test = text_vector(title_test)
input_label_test = tf.convert_to_tensor(label_test)

In [ ]:
test_loss, test_acc = model.evaluate(input_title_test, input_label_test)

In [ ]:
print(test_loss, test_acc)